In [1]:
%%init_spark
launcher.master="yarn"
launcher.num_executors=6
launcher.executor_cores=2
launcher.executor_memory='2600m'
launcher.packages=["com.github.master:spark-stemming_2.10:0.2.0"]

load the review.json file  and extract “text” and “stars” attributes

find the distribution of “stars”
attributes; that is,
find the number of reviews for 
each star value

In [2]:
val reviewsDF=spark.read.json("/hadoop-user/review.json").select("Text","stars")
reviewsDF.show(3)
reviewsDF.cache()

Intitializing Scala interpreter ...

Spark Web UI available at http://bd-hm18:8088/proxy/application_1543027522566_0001
SparkContext available as 'sc' (version = 2.3.2, master = yarn, app id = application_1543027522566_0001)
SparkSession available as 'spark'


2018-11-23 20:45:47 WARN  NativeCodeLoader:62 - Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
2018-11-23 20:45:49 WARN  Utils:66 - Service 'SparkUI' could not bind on port 4040. Attempting port 4041.
2018-11-23 20:45:51 WARN  Client:66 - Neither spark.yarn.jars nor spark.yarn.archive is set, falling back to uploading libraries under SPARK_HOME.
2018-11-23 20:46:00 WARN  Client:66 - Same path resource file:///home/administrator/.ivy2/jars/com.github.master_spark-stemming_2.10-0.2.0.jar added multiple times to distributed cache.
+--------------------+-----+
|                Text|stars|
+--------------------+-----+
|The pizza was oka...|    2|
|I love this place...|    5|
|Terrible. Dry cor...|    1|
+--------------------+-----+
only showing top 3 rows



reviewsDF: org.apache.spark.sql.DataFrame = [Text: string, stars: bigint]
res0: reviewsDF.type = [Text: string, stars: bigint]


In [3]:
reviewsDF.createOrReplaceTempView("reviews")
spark.sql("select stars,count(stars) from reviews group by stars").show()

+-----+------------+
|stars|count(stars)|
+-----+------------+
|    5|         361|
|    1|         117|
|    3|         173|
|    2|          85|
|    4|         264|
+-----+------------+



In [4]:
import org.apache.spark.ml.feature._
import org.apache.spark.ml.linalg._

import org.apache.spark.ml.feature._
import org.apache.spark.ml.linalg._


In [5]:
val splits = Array(Double.NegativeInfinity, 3,Double.PositiveInfinity)

val  ratingsData= spark.read.json("/hadoop-user/review.json").select("stars")

ratingsData.show(10)

+-----+
|stars|
+-----+
|    2|
|    5|
|    1|
|    2|
|    5|
|    1|
|    5|
|    5|
|    4|
|    2|
+-----+
only showing top 10 rows



splits: Array[Double] = Array(-Infinity, 3.0, Infinity)
ratingsData: org.apache.spark.sql.DataFrame = [stars: bigint]


In [6]:
val splits = Array(Double.NegativeInfinity, 4,Double.PositiveInfinity)


//val  ratingsData= spark.read.json("/hadoop-user/review.json").select("stars")

val  ratingsData= spark.read.json("/review.json").select("stars")

val bucketizer = new Bucketizer()
  .setInputCol("stars")
  .setOutputCol("stars_bucketed")
  .setSplits(splits)

// Transform original data into its bucket index.
val bucketedData = bucketizer.transform(ratingsData)
bucketedData.show
// Transform original data into its bucket index.
//val bucketedData = bucketizer.transform(ratingsDF)
//bucketedData.show

+-----+--------------+
|stars|stars_bucketed|
+-----+--------------+
|    5|           1.0|
|    5|           1.0|
|    5|           1.0|
|    5|           1.0|
|    4|           1.0|
|    4|           1.0|
|    5|           1.0|
|    4|           1.0|
|    4|           1.0|
|    3|           0.0|
|    5|           1.0|
|    4|           1.0|
|    3|           0.0|
|    1|           0.0|
|    3|           0.0|
|    5|           1.0|
|    4|           1.0|
|    1|           0.0|
|    3|           0.0|
|    3|           0.0|
+-----+--------------+
only showing top 20 rows



splits: Array[Double] = Array(-Infinity, 4.0, Infinity)
ratingsData: org.apache.spark.sql.DataFrame = [stars: bigint]
bucketizer: org.apache.spark.ml.feature.Bucketizer = bucketizer_2468f59185fc
bucketedData: org.apache.spark.sql.DataFrame = [stars: bigint, stars_bucketed: double]


In [7]:
//val bucketedData = bucketizer.transform(ratingsData)
//bucketedData.show
bucketedData
    .groupBy("stars_bucketed")
    .count()
    .show()

+--------------+-------+
|stars_bucketed|  count|
+--------------+-------+
|           0.0|1785005|
|           1.0|3476664|
+--------------+-------+



In [55]:

  // Create a fraction map where we are only interested:
  // - 50% of the rows that have answer_count = 5
  // - 10% of the rows that have answer_count = 10
  // - 100% of the rows that have answer_count = 20
  // Note also that fractions should be in the range [0, 1]
//println(test);

//1785005/3476664 = 0.513424 

//it is returing zero since it is a int, so as a workaround manually putting the decimal value
//else we get zero
//will convert to double if time permits
  val fractionKeyMap = Map(0.0 -> 0.1, 1.0 -> 0.1*0.513424)

  // Stratified sample using the fractionKeyMap.
  bucketedData
    .stat
    .sampleBy("stars_bucketed", fractionKeyMap, 111L)
    .groupBy("stars_bucketed")
    .count()
    .show()

+--------------+------+
|stars_bucketed| count|
+--------------+------+
|           0.0|178418|
|           1.0|178580|
+--------------+------+



fractionKeyMap: scala.collection.immutable.Map[Double,Double] = Map(0.0 -> 0.1, 1.0 -> 0.0513424)



Extract TFIDF vectors from 
the review Text. 
When creating countVectorizer, use 
setMinDF(100) 
to only include words in the feature vector  that appear in at 
least 100 
reviews.
Make sure that you remove stop words and punctuations and 
use stemming as explained in the labs

In [20]:
launcher.packages=["com.github.master:spark-stemming_2.10:0.2.0"]

<console>: 2: error: illegal start of simple expression

In [19]:
import org.apache.spark.mllib.feature.Stemmer


<console>: 30: error: object Stemmer is not a member of package org.apache.spark.mllib.feature

In [21]:
//val reviewsDF=spark.read.json("/review.json").select("Text")
val reviewsDF=spark.read.json("/hadoop-user/review.json").select("Text")

import org.apache.spark.mllib.feature.Stemmer


reviewsDF.cache()

import org.apache.spark.ml.feature._
val tokenized = new RegexTokenizer().setMinTokenLength(2).setToLowercase(true).setInputCol("Text").setOutputCol("words").
transform(reviewsDF)


//Defining a udf to remove punctuations from a sequence of words
import org.apache.spark.sql.functions.udf

def removePunc(words:Seq[String]):Seq[String]={
 return words.map(_.replaceAll("\\p{Punct}",""))
}

val removePuncUDF=udf(removePunc(_:Seq[String]))

//use the removePuncUDF to remove all punctuations from words
val reviewWords= tokenized.withColumn("words",removePuncUDF($"words"))

//println("A sample of the tokenized reviews after removing punctuations")
//reviewWords.show(5, truncate=55)

val reviews_filtered=new StopWordsRemover().setInputCol("words").setOutputCol("filtered_words").transform(reviewWords).select("Text","filtered_words")
//println("reviews after removing stop words")
//reviews_filtered.show(2, truncate=55)


//Stemming using a third party package
import org.apache.spark.mllib.feature.Stemmer
val stemmed_reviews = new Stemmer().setInputCol("filtered_words").setOutputCol("stemmed_words").transform(reviews_filtered)
println("After stemming")
stemmed_reviews.select("Text","stemmed_words").show(2, truncate=60)

val bow = new CountVectorizer().setInputCol("stemmed_words").setOutputCol("BOW").setMinDF(100).fit(stemmed_reviews).transform(stemmed_reviews)
bow.show(3)
reviewsDF.unpersist()
bow.cache()

val tfidf = new IDF().setInputCol("BOW").setOutputCol("TFIDF features").fit(bow).transform(bow)
println("A sample of TFIDF feature vectors")
tfidf.select("TFIDF features").show(5,truncate=70)


<console>: 35: error: object Stemmer is not a member of package org.apache.spark.mllib.feature